In [ ]:
# Lab 3: 고급 검색 - 화재예방 가이드 (40분)
# 소방청 민원처리 AI Agent 교육

"""
학습 목표:
- 다중 문서 소스 관리
- 소방기본법 + 화재예방 가이드 통합 검색
- 검색 결과 병합 및 활용
"""

# ============================================================
# 환경 설정
# ============================================================
import sys,os
if 'google.colab' in sys.modules:
    !pip install openai chromadb pdfplumber -q

from openai import OpenAI
import pdfplumber
import chromadb
from chromadb.config import Settings
from typing import List, Dict

# API 키 설정
print("[API 키 입력] OpenAI API 키를 입력하세요")
api_key = "".strip()
client = OpenAI(api_key=api_key)


print(" 환경 설정 완료!\n")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.7/20.7 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 109.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.0 MB/s eta 0:0

In [2]:
# ============================================================
# Lab 2 함수 재사용
# ============================================================

def extract_text_from_pdf(pdf_path: str, max_pages: int = None) -> List[Dict]:
    """PDF에서 텍스트 추출"""
    pages_data = []

    with pdfplumber.open(pdf_path) as pdf:
        total_pages = len(pdf.pages) if max_pages is None else min(max_pages, len(pdf.pages))

        for i in range(total_pages):
            page = pdf.pages[i]
            text = page.extract_text()

            pages_data.append({
                "page_num": i + 1,
                "text": text,
                "char_count": len(text)
            })

    return pages_data

def chunk_text(pages_data: List[Dict], chunk_size: int = 500, overlap: int = 50) -> List[Dict]:
    """텍스트 청킹"""
    chunks = []
    chunk_id = 0

    for page_data in pages_data:
        text = page_data['text']
        page_num = page_data['page_num']

        start = 0
        while start < len(text):
            end = start + chunk_size
            chunk_text = text[start:end]

            if chunk_text.strip():
                chunks.append({
                    "id": f"chunk_{chunk_id}",
                    "text": chunk_text,
                    "page": page_num,
                    "start_pos": start
                })
                chunk_id += 1

            start = end - overlap

    return chunks

def get_embedding(text: str) -> List[float]:
    """임베딩 생성"""
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=text
    )
    return response.data[0].embedding



In [4]:
# ============================================================
# 실습 1: 소방기본법 Vector DB 재구축
# ============================================================

print("\n" + "="*60)
print("[실습 1] 소방기본법 Vector DB 재구축")
print("="*60)

# ChromaDB 초기화
print("\n[진행중] ChromaDB 초기화...")
chroma_client = chromadb.Client(Settings(
    persist_directory="./chroma_db",
    anonymized_telemetry=False
))

# 기존 컬렉션 삭제
try:
    chroma_client.delete_collection("fire_law")
    chroma_client.delete_collection("fire_prevention")
except:
    pass

# 소방기본법 처리
print("\n[진행중] 소방기본법.pdf 처리...")
fire_law_pages = extract_text_from_pdf("소방기본법.pdf", max_pages=10)
fire_law_chunks = chunk_text(fire_law_pages, chunk_size=500, overlap=50)
print(f"[완료] {len(fire_law_chunks)}개 청크 생성")

# 컬렉션 생성 및 저장
fire_law_collection = chroma_client.create_collection(
    name="fire_law",
    metadata={"description": "소방기본법"}
)

print("[진행중] 소방기본법 임베딩 및 저장...")
for i, chunk in enumerate(fire_law_chunks):
    if i % 10 == 0 and i > 0:
        print(f"  진행: {i}/{len(fire_law_chunks)}")

    embedding = get_embedding(chunk['text'])

    fire_law_collection.add(
        ids=[f"law_{chunk['id']}"],
        embeddings=[embedding],
        documents=[chunk['text']],
        metadatas=[{
            "page": chunk['page'],
            "source": "소방기본법"
        }]
    )

print(f"[완료] 소방기본법 {len(fire_law_chunks)}개 청크 저장 완료")




[실습 1] 소방기본법 Vector DB 재구축

[진행중] ChromaDB 초기화...

[진행중] 소방기본법.pdf 처리...
[완료] 26개 청크 생성
[진행중] 소방기본법 임베딩 및 저장...
  진행: 10/26
  진행: 20/26
[완료] 소방기본법 26개 청크 저장 완료


In [5]:
# ============================================================
# 실습 2: 화재예방 가이드 Vector DB 구축
# ============================================================

print("\n\n" + "="*60)
print("[실습 2] 화재예방 가이드 Vector DB 구축")
print("="*60)

# 화재예방 PDF 처리
print("\n[진행중] 화재예방.pdf 처리...")
prevention_pages = extract_text_from_pdf("화재예방.pdf")
print(f"[완료] 총 {len(prevention_pages)} 페이지 추출")

prevention_chunks = chunk_text(prevention_pages, chunk_size=500, overlap=50)
print(f"[완료] {len(prevention_chunks)}개 청크 생성")

# 컬렉션 생성
prevention_collection = chroma_client.create_collection(
    name="fire_prevention",
    metadata={"description": "화재예방가이드"}
)

# 임베딩 및 저장
print("\n[진행중] 화재예방 가이드 임베딩 및 저장...")
print("(시간이 좀 걸립니다 - 약 2-3분)")

for i, chunk in enumerate(prevention_chunks):
    if i % 20 == 0 and i > 0:
        print(f"  진행: {i}/{len(prevention_chunks)}")

    embedding = get_embedding(chunk['text'])

    prevention_collection.add(
        ids=[f"prev_{chunk['id']}"],
        embeddings=[embedding],
        documents=[chunk['text']],
        metadatas=[{
            "page": chunk['page'],
            "source": "화재예방가이드"
        }]
    )

print(f"[완료] 화재예방 가이드 {len(prevention_chunks)}개 청크 저장 완료")





[실습 2] 화재예방 가이드 Vector DB 구축

[진행중] 화재예방.pdf 처리...
[완료] 총 44 페이지 추출
[완료] 193개 청크 생성

[진행중] 화재예방 가이드 임베딩 및 저장...
(시간이 좀 걸립니다 - 약 2-3분)
  진행: 20/193
  진행: 40/193
  진행: 60/193
  진행: 80/193
  진행: 100/193
  진행: 120/193
  진행: 140/193
  진행: 160/193
  진행: 180/193
[완료] 화재예방 가이드 193개 청크 저장 완료


In [6]:
# ============================================================
# 실습 3: 개별 검색 함수
# ============================================================

print("\n\n" + "="*60)
print("[실습 3] 개별 검색 함수 구현")
print("="*60)

def search_fire_law(query: str, n_results: int = 3) -> List[Dict]:
    """소방기본법 검색"""
    query_embedding = get_embedding(query)

    results = fire_law_collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results
    )

    search_results = []
    for i in range(len(results['documents'][0])):
        search_results.append({
            "text": results['documents'][0][i],
            "page": results['metadatas'][0][i]['page'],
            "source": results['metadatas'][0][i]['source']
        })

    return search_results

def search_fire_prevention(query: str, n_results: int = 3) -> List[Dict]:
    """화재예방 가이드 검색"""
    query_embedding = get_embedding(query)

    results = prevention_collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results
    )

    search_results = []
    for i in range(len(results['documents'][0])):
        search_results.append({
            "text": results['documents'][0][i],
            "page": results['metadatas'][0][i]['page'],
            "source": results['metadatas'][0][i]['source']
        })

    return search_results

# 개별 검색 테스트
print("\n[테스트] 개별 검색")
test_query = "소화기 점검"

print(f"\n[검색어] {test_query}")
print("\n[소방기본법 검색 결과]")
law_results = search_fire_law(test_query, n_results=2)
for i, r in enumerate(law_results, 1):
    print(f"  {i}. {r['page']}페이지: {r['text'][:100]}...")

print("\n[화재예방 가이드 검색 결과]")
prev_results = search_fire_prevention(test_query, n_results=2)
for i, r in enumerate(prev_results, 1):
    print(f"  {i}. {r['page']}페이지: {r['text'][:100]}...")





[실습 3] 개별 검색 함수 구현

[테스트] 개별 검색

[검색어] 소화기 점검

[소방기본법 검색 결과]
  1. 4페이지: 「소방기본법」
소방기본법
[시행 2024. 7. 31.] [법률 제20156호, 2024. 1. 30., 일부개정]
소방청 (대응총괄과-총괄,소방활동,소방력동원) 044-205-7...
  2. 8페이지: 다. 다만, 「수도법」 제45조에 따라 소화전을 설치하는 일반수도사업자는 관할 소
방서장과 사전협의를 거친 후 소화전을 설치하여야 하며, 설치 사실을 관할 소방서장에게 통
지하고,...

[화재예방 가이드 검색 결과]
  1. 20페이지:  연기를 신청하려는 관계인은 행정안전부령으로 정하는 바에 따라
연기신청서에 연기의 사유 및 기간 등을 적어 소방관서장에게 제출해야 한다.
③ 소방관서장은 법 제8조제4항 후단에 따...
  2. 4페이지: 화재의 예방 및 안전관리에 관한 법률
제13조(화재안전조사 결과 통보) 소방관서장은 화재안전조사를 마친 때에는 그 조사 결과를 관계인에게 서면으로 통지
하여야 한다. 다만, 화재안...


In [7]:
# ============================================================
# 실습 4: 통합 검색 시스템
# ============================================================

print("\n\n" + "="*60)
print("[실습 4] 통합 검색 시스템 - 두 소스 동시 검색")
print("="*60)

def search_both_sources(query: str, n_results_per_source: int = 2) -> Dict:
    """
    소방기본법 + 화재예방 가이드 동시 검색

    장점:
    1. 법령과 실무 가이드를 함께 참고 가능
    2. 더 풍부한 컨텍스트 제공
    3. 법적 근거 + 실무 방안 동시 제시
    """
    law_results = search_fire_law(query, n_results_per_source)
    prevention_results = search_fire_prevention(query, n_results_per_source)

    return {
        "법령": law_results,
        "예방가이드": prevention_results,
        "total_count": len(law_results) + len(prevention_results)
    }





[실습 4] 통합 검색 시스템 - 두 소스 동시 검색


In [8]:
# ============================================================
# 실습 5: 통합 검색 테스트
# ============================================================

print("\n[실습 5] 통합 검색 시스템 테스트")
print("="*60)

test_queries = [
    "소화기 관리 방법",
    "화재 발생 시 대응 절차",
    "비상구 설치 기준"
]

for query in test_queries:
    print(f"\n[검색어] {query}")
    print("-"*50)

    combined_results = search_both_sources(query, n_results_per_source=2)

    print("\n[소방기본법 - 법적 근거]")
    for i, r in enumerate(combined_results['법령'], 1):
        print(f"  {i}. {r['page']}페이지")
        print(f"     {r['text'][:120]}...")

    print("\n[화재예방가이드 - 실무 가이드]")
    for i, r in enumerate(combined_results['예방가이드'], 1):
        print(f"  {i}. {r['page']}페이지")
        print(f"     {r['text'][:120]}...")

    print("\n" + "="*60)




[실습 5] 통합 검색 시스템 테스트

[검색어] 소화기 관리 방법
--------------------------------------------------

[소방기본법 - 법적 근거]
  1. 4페이지
     「소방기본법」
소방기본법
[시행 2024. 7. 31.] [법률 제20156호, 2024. 1. 30., 일부개정]
소방청 (대응총괄과-총괄,소방활동,소방력동원) 044-205-7562,7572
소방청 (119종합상...
  2. 8페이지
     다. 다만, 「수도법」 제45조에 따라 소화전을 설치하는 일반수도사업자는 관할 소
방서장과 사전협의를 거친 후 소화전을 설치하여야 하며, 설치 사실을 관할 소방서장에게 통
지하고, 그 소화전을 유지ㆍ관리하여야 한다....

[화재예방가이드 - 실무 가이드]
  1. 4페이지
     화재의 예방 및 안전관리에 관한 법률
제13조(화재안전조사 결과 통보) 소방관서장은 화재안전조사를 마친 때에는 그 조사 결과를 관계인에게 서면으로 통지
하여야 한다. 다만, 화재안전조사의 현장에서 관계인에게 조사의 ...
  2. 34페이지
     화재의 예방 및 안전관리에 관한 법률 시행규칙
2. 증축 또는 용도변경으로 인하여 특정소방대상물이 영 제25조제1항에 따른 소방안전관리대상물로 된 경우 또는
특정소방대상물의 소방안전관리 등급이 변경된 경우: 증축공사...


[검색어] 화재 발생 시 대응 절차
--------------------------------------------------

[소방기본법 - 법적 근거]
  1. 8페이지
     로 수행하기 어려운 화재, 재난ㆍ재해, 그 밖의 구조ㆍ구급이 필요한 상황이 발생하거나 특별
히 국가적 차원에서 소방활동을 수행할 필요가 인정될 때에는 각 시ㆍ도지사에게 행정안전부
법제처 6 국가법령정보센터...
  2. 9페이지
     
상황이 발생하였을 때에는 소방대를 현장에 신속하게 출동시켜 화재진압과 인명구조ㆍ구급
등 소방에 필요한 활동(이하 이 조에서 “소방활동”이라

In [9]:
# ============================================================
# 실습 6: 통합 답변 생성
# ============================================================

print("\n\n" + "="*60)
print("[실습 6] 통합 RAG 답변 생성")
print("="*60)

def answer_with_both_sources(question: str) -> str:
    """
    양쪽 소스를 활용한 답변 생성
    """
    # 통합 검색
    results = search_both_sources(question, n_results_per_source=2)

    # 컨텍스트 구성
    law_context = "\n".join([
        f"- {r['page']}페이지: {r['text'][:200]}"
        for r in results['법령']
    ])

    prevention_context = "\n".join([
        f"- {r['page']}페이지: {r['text'][:200]}"
        for r in results['예방가이드']
    ])

    # LLM 답변 생성
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": """당신은 소방 전문가입니다.
소방기본법(법령)과 화재예방 가이드(실무)를 모두 참고하여 답변하세요.
1. 먼저 법적 근거를 제시하고
2. 실무적인 예방 방법을 안내하세요."""
            },
            {
                "role": "user",
                "content": f"""다음 자료를 참고하여 질문에 답하세요.

[소방기본법 - 법적 근거]
{law_context}

[화재예방 가이드 - 실무 방안]
{prevention_context}

[질문]
{question}"""
            }
        ],
        temperature=0.4,
        max_tokens=600
    )

    return response.choices[0].message.content

# 테스트
print("\n[테스트] 통합 답변 생성")
test_question = "음식점 주방에서 화재 예방을 위해 어떤 조치가 필요한가요?"
print(f"\n[질문] {test_question}")
print("-"*50)
print("\n[답변]")
print(answer_with_both_sources(test_question))





[실습 6] 통합 RAG 답변 생성

[테스트] 통합 답변 생성

[질문] 음식점 주방에서 화재 예방을 위해 어떤 조치가 필요한가요?
--------------------------------------------------

[답변]
음식점 주방에서 화재 예방을 위해 필요한 조치는 다음과 같이 법적 근거와 실무적인 예방 방법을 기반으로 안내하겠습니다.

### 1. 법적 근거
소방기본법 제9조에 따르면, 소방대는 화재가 발생했을 때 신속하게 출동하여 화재진압 및 인명 구조 등의 소방활동을 수행해야 하며, 누구든지 정당한 사유 없이 소방대의 활동을 방해해서는 안 됩니다. 이는 화재 예방 및 안전 관리의 중요성을 강조합니다.

또한, 화재 예방 가이드에서는 화기 취급 시 안전요원이 배치되어야 하며, 소방관서장과 사전 협의하여 안전조치를 취해야 한다고 명시하고 있습니다. 이는 음식점 주방에서 화재를 예방하기 위한 필수적인 절차입니다.

### 2. 실무적인 예방 방법
음식점 주방에서 화재를 예방하기 위한 실무적인 방법은 다음과 같습니다:

1. **정기적인 점검 및 유지보수**: 주방의 모든 전기 및 가스 설비를 정기적으로 점검하고, 문제가 발견되면 즉시 수리합니다. 특히, 배선과 가스관의 누출 여부를 확인해야 합니다.

2. **화재 감시자 및 안전요원 배치**: 주방에서 화기를 취급하는 경우, 화재 감시자나 안전요원을 배치하여 화재 발생 시 즉시 대응할 수 있도록 합니다.

3. **안전조치 협의**: 주방에서 특별한 화기 사용이 필요한 경우, 소방관서장과 사전 협의하여 화재 예방 안전조치 협의 신청서를 제출하고, 필요한 안전조치를 이행합니다.

4. **소화기 및 소화전 비치**: 주방 내에 적절한 용량의 소화기를 비치하고, 소화전이 있는 경우 그 위치를 명확히 하여 모든 직원이 쉽게 접근할 수 있도록 합니다. 소화기의 사용법에 대한 교육도 정기적으로 실시합니다.

5. **화재 예방 교육**: 모든 직원에게 정기적인 화재 예방 교육을 실시하여, 화재 발

In [10]:
# ============================================================
# Lab 3 완료
# ============================================================

print("\n\n" + "="*60)
print("[완료] Lab 3 완료!")
print("="*60)
print("""
학습 내용:
1. 다중 문서 소스 관리 (2개 Vector DB)
2. 개별 검색 함수 구현
3. 통합 검색 시스템 구축
4. 법령 + 가이드 융합 답변 생성

핵심 함수:
- search_fire_law(): 소방기본법 검색
- search_fire_prevention(): 화재예방 검색
- search_both_sources(): 통합 검색 (Lab 4에서 사용)
- answer_with_both_sources(): 통합 답변

다음 Lab:
- Lab 4에서는 이 통합 검색을 활용하여 민원 처리 보고서를 자동 생성합니다.
""")



[완료] Lab 3 완료!

학습 내용:
1. 다중 문서 소스 관리 (2개 Vector DB)
2. 개별 검색 함수 구현
3. 통합 검색 시스템 구축
4. 법령 + 가이드 융합 답변 생성

핵심 함수:
- search_fire_law(): 소방기본법 검색
- search_fire_prevention(): 화재예방 검색
- search_both_sources(): 통합 검색 (Lab 4에서 사용)
- answer_with_both_sources(): 통합 답변

다음 Lab:
- Lab 4에서는 이 통합 검색을 활용하여 민원 처리 보고서를 자동 생성합니다.

